In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ZLPKoMvyRwO3jn9eeAD8Ug',
 'name': '310ad8ef32ea',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


# SQL search

In [2]:
import os
import json

es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

operations = []
documents = json.load(open(os.path.join(os.getcwd(), "data", "dummy_knn.json")))

for document in documents:
    operations.append({'index': {'_index': 'my_index'}})
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

{'errors': False,
 'items': [{'index': {'_id': 'H8cwNZMBoO_Kuf5q9iA7',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'IMcwNZMBoO_Kuf5q9iA7',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'IccwNZMBoO_Kuf5q9iA7',
                      '_index': 'my_index',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed': 0, 'successful': 1, '

In [3]:
count = es.count(index='my_index')
print('Number of documents in index:', count.body['count'])

Number of documents in index: 10


In [4]:
query = {
    "query": "SELECT title FROM my_index ORDER BY id LIMIT 5"
}

response = es.sql.query(body=query)
for row in response['rows']:
    print(row)

['The Solar System']
['Black Holes']
['Galaxies']
['The Big Bang Theory']
['Exoplanets']


In [10]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='json')
pprint(response['rows'])

[['The Solar System consists of the Sun and the objects that orbit it, '
  'including eight planets, their moons, dwarf planets, and countless small '
  'bodies like asteroids and comets.',
  1,
  'The Solar System'],
 ['A black hole is a region of space where the gravitational pull is so strong '
  'that nothing, not even light, can escape from it. They are formed when '
  'massive stars collapse under their own gravity.',
  2,
  'Black Holes'],
 ['Galaxies are vast systems that consist of stars, stellar remnants, '
  'interstellar gas, dust, and dark matter. The Milky Way is the galaxy that '
  'contains our Solar System.',
  3,
  'Galaxies'],
 ['The Big Bang Theory is the leading explanation about how the universe '
  'began. It suggests that the universe was once in an extremely hot and dense '
  'state and has been expanding ever since.',
  4,
  'The Big Bang Theory'],
 ['Exoplanets, or extrasolar planets, are planets that exist outside our solar '
  'system. They vary greatly in 

In [11]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='txt')
print(response)

                                                                                                        content                                                                                                         |      id       |         title         
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------------
The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.                                          |1              |The Solar System       
A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.                      |2           

In [12]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='csv')
print(response)

content,id,title
"The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.",1,The Solar System
"A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.",2,Black Holes
"Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.",3,Galaxies
The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.,4,The Big Bang Theory
"Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method a

In [13]:
query = {
    "query": "SELECT * FROM my_index",
}

response = es.sql.query(body=query, format='tsv')
print(response)

content	id	title
The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless small bodies like asteroids and comets.	1	The Solar System
A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.	2	Black Holes
Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.	3	Galaxies
The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.	4	The Big Bang Theory
Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method and radi

In [17]:
query = {
    "query": "SELECT * FROM my_index"
}

response = es.sql.query(
    body=query,
    filter={
        "term": {
            "title.keyword": "Black Holes"
        }
    },
)
pprint(response['rows'])

[['A black hole is a region of space where the gravitational pull is so strong '
  'that nothing, not even light, can escape from it. They are formed when '
  'massive stars collapse under their own gravity.',
  2,
  'Black Holes']]


C:\Users\lexua\AppData\Local\Temp\ipykernel_16380\934601633.py:5: DeprecationWarning: Received 'filter' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.sql.query(


In [18]:
query = {
    "query": "SELECT * FROM my_index ORDER BY id DESC"
}

response = es.sql.query(
    body=query,
    format='json',
    fetch_size=5,
)
response.body

C:\Users\lexua\AppData\Local\Temp\ipykernel_16380\3586786142.py:5: DeprecationWarning: Received 'fetch_size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.sql.query(


{'columns': [{'name': 'content', 'type': 'text'},
  {'name': 'id', 'type': 'long'},
  {'name': 'title', 'type': 'text'}],
 'rows': [['Space exploration involves the use of space technology to explore outer space. It includes missions to planets, moons, and other celestial bodies, aiming to discover more about the universe.',
   10,
   'Space Exploration'],
  ['The universe has been expanding since the Big Bang. Observations of distant galaxies show that they are moving away from us, which supports the idea of an expanding universe.',
   9,
   'The Expanding Universe'],
  ["Dark matter is a type of matter that does not emit light or energy. It cannot be observed directly but is believed to make up about 27% of the universe's total mass and energy.",
   8,
   'Dark Matter'],
  ['Astrobiology is the study of the origin, evolution, distribution, and future of life in the universe. It combines elements of biology, chemistry, and planetary science.',
   7,
   'Astrobiology'],
  ['Stars are b

In [19]:
while 'cursor' in response.body:
    response = es.sql.query(
        format='json',
        cursor=response.body['cursor'],
    )
    print(response.body)
    print()

{'rows': [['Exoplanets, or extrasolar planets, are planets that exist outside our solar system. They vary greatly in size and composition and are often found using methods like the transit method and radial velocity.', 5, 'Exoplanets'], ['The Big Bang Theory is the leading explanation about how the universe began. It suggests that the universe was once in an extremely hot and dense state and has been expanding ever since.', 4, 'The Big Bang Theory'], ['Galaxies are vast systems that consist of stars, stellar remnants, interstellar gas, dust, and dark matter. The Milky Way is the galaxy that contains our Solar System.', 3, 'Galaxies'], ['A black hole is a region of space where the gravitational pull is so strong that nothing, not even light, can escape from it. They are formed when massive stars collapse under their own gravity.', 2, 'Black Holes'], ['The Solar System consists of the Sun and the objects that orbit it, including eight planets, their moons, dwarf planets, and countless sm

In [20]:
translate_query = {
    "query": "SELECT * FROM my_index WHERE content LIKE '%universe%' ORDER BY id DESC LIMIT 20"
}

translated_query = es.sql.translate(body=translate_query)
translated_query.body

{'size': 20,
 'query': {'wildcard': {'content.keyword': {'wildcard': '*universe*',
    'boost': 1.0}}},
 '_source': False,
 'fields': [{'field': 'content'}, {'field': 'id'}, {'field': 'title'}],
 'sort': [{'id': {'order': 'desc',
    'missing': '_first',
    'unmapped_type': 'long'}}],
 'track_total_hits': -1}